In [1]:
!pip install ctgan
!pip install sdv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from ctgan import CTGAN 

import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

dataset = pd.read_csv('https://raw.githubusercontent.com/mahayasa/various-sampling-churn-prediction/main/data/mobile-churn.csv')


df1=dataset[dataset['churn']==0]
df=dataset[dataset['churn']==1]

#filling missing value with mean
df = df.fillna(df.mean())
df=df.drop(['year','month','user_account_id'],axis=1)

#filling missing value with mean
df1 = df1.fillna(df1.mean())
df1=df1.drop(['year','month','user_account_id'],axis=1)

batch_size = 5000
epochs = 100
model = CTGAN(batch_size=batch_size, epochs=epochs, verbose=True)
model.fit(df)

Epoch 1, Loss G:  0.1658,Loss D: -0.2143
Epoch 2, Loss G:  0.4464,Loss D: -0.7390
Epoch 3, Loss G:  0.7826,Loss D: -1.4588
Epoch 4, Loss G:  1.2082,Loss D: -2.3476
Epoch 5, Loss G:  1.5510,Loss D: -3.2544
Epoch 6, Loss G:  1.9403,Loss D: -4.2348
Epoch 7, Loss G:  2.0740,Loss D: -5.0813
Epoch 8, Loss G:  2.0537,Loss D: -5.6296
Epoch 9, Loss G:  1.6537,Loss D: -5.5659
Epoch 10, Loss G:  0.9660,Loss D: -5.3799
Epoch 11, Loss G:  0.4012,Loss D: -4.6481
Epoch 12, Loss G: -0.3049,Loss D: -3.9198
Epoch 13, Loss G: -1.0325,Loss D: -3.3491
Epoch 14, Loss G: -1.6833,Loss D: -2.3887
Epoch 15, Loss G: -2.4117,Loss D: -1.7630
Epoch 16, Loss G: -3.0238,Loss D: -1.3594
Epoch 17, Loss G: -3.4488,Loss D: -0.7995
Epoch 18, Loss G: -3.8021,Loss D: -0.5720
Epoch 19, Loss G: -3.9888,Loss D: -0.7477
Epoch 20, Loss G: -4.2901,Loss D: -0.5863
Epoch 21, Loss G: -4.5431,Loss D: -0.4535
Epoch 22, Loss G: -4.4499,Loss D: -0.4877
Epoch 23, Loss G: -4.5211,Loss D: -0.2967
Epoch 24, Loss G: -4.5573,Loss D: -0.2019
E

In [3]:
n_generated_data = 38655
generated_df = model.sample(n_generated_data)

In [4]:
#concat original data and gan data
data_concat = pd.concat([df, generated_df])
# combine data churn and not churn
data=pd.concat([df1, data_concat])
data

,user_lifetime,user_intake,user_no_outgoing_activity_in_days,user_account_balance_last,user_spendings,user_has_outgoing_calls,user_has_outgoing_sms,user_use_gprs,user_does_reload,reloads_inactive_days,...,last_100_calls_outgoing_duration,last_100_calls_outgoing_to_onnet_duration,last_100_calls_outgoing_to_offnet_duration,last_100_calls_outgoing_to_abroad_duration,last_100_sms_outgoing_count,last_100_sms_outgoing_to_onnet_count,last_100_sms_outgoing_to_offnet_count,last_100_sms_outgoing_to_abroad_count,last_100_gprs_usage,churn
0,1000,0,1,0.050000,0.000000,1,1,0,0,66,...,75.270000,0.000000,63.430000,0.000000,210,1,84,0,0.000000,0
1,1000,0,25,28.310000,3.450000,1,0,0,0,1276,...,13.380000,11.180000,2.000000,11.180000,0,0,0,0,0.000000,0
2,1005,0,8,15.620000,1.970000,1,0,0,0,1276,...,30.000000,0.000000,0.000000,10.450000,0,0,0,0,0.000000,0
4,1032,0,2,5.860000,0.150000,1,0,0,1,17,...,2.580000,0.000000,1.000000,0.000000,0,0,0,0,0.000000,0
5,1039,0,1,12.510000,12.960000,1,1,0,1,6,...,356.650000,0.000000,215.830000,0.000000,4,1,1,0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38650,15721,0,1302,-0.135776,0.078555,1,0,0,1,1275,...,-1.634881,0.005846,-1.394899,-0.084209,1,0,606,0,-0.755039,1
38651,-241,0,57,-0.020193,0.230304,1,0,0,1,1296,...,0.213788,-0.080823,1.853549,-0.064908,-74,0,1,0,0.858652,1
38652,15825,0,8,0.217205,1.160032,0,1,0,1,1241,...,1.010488,0.062754,0.537410,1.858218,54,0,30,0,-0.111447,1
38653,15821,0,4,-0.228061,0.121949,0,0,0,0,1249,...,-0.213237,-0.044174,1.044904,-0.006101,-2,0,137,0,-0.553252,1


In [8]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun  1 17:31:40 2020

@author: manav

Modifed on 23 AUG 2022

by mahayasa adiputra
"""

import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
import sklearn.metrics as mt
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from numpy import mean
from numpy import std

#data=data.dropna()
#data=data.fillna(0)

X=data.drop(['churn'],axis=1)
y=data["churn"]

#enn = EditedNearestNeighbours(n_neighbors=3)
#X, y = enn.fit_resample(X, y)
#ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
#X, y = ncr.fit_resample(X, y)
#tomek_links = TomekLinks()
#X, y = tomek_links.fit_resample(X, y)


#kfold cross validation
cv = KFold(n_splits=5, random_state=1, shuffle=True)


#Training XGBoost
#classifier = XGBClassifier(eta=0.3, max_depth = 4, gamma=0, min_child_weight=1)
classifier = XGBClassifier(colsample_bytree=0.6, max_depth = 5, gamma=1.5, min_child_weight=5)

score=cross_val_score(classifier, X, y, scoring='f1', cv=cv, n_jobs=-1)
print('===============XGBoost Performance====================')

print('F1 score: %.3f' % (mean(score)))
print('STD F1 Score: %.3f' % (std(score)))

rc=cross_val_score(classifier, X, y, scoring='recall', cv=cv, n_jobs=-1)
print('Recall: %.3f' % (mean(rc)))

from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score

# Define the specificity scorer
scorer = make_scorer(specificity_score)

# Calculate the cross-validated specificity score
sp = cross_val_score(classifier, X, y, cv=5, scoring=scorer)
print('Specitifity: %.3f' % (mean(sp)))

auc=cross_val_score(classifier, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('AUC ROC: %.3f' % (mean(auc)))

import math

gmean=rc*sp
gmean=mean(gmean)
sqrtg = math.sqrt(gmean)
print('G-Mean: %.3f' % (sqrtg))
print('======================================================')

===============XGBoost Performance====================
F1 score: 0.921
STD F1 Score: 0.001
Recall: 0.915
Specitifity: 0.847
AUC ROC: 0.975
G-Mean: 0.881


In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators =100, criterion = 'gini', random_state = 0)

score=cross_val_score(classifier, X, y, scoring='f1', cv=cv, n_jobs=-1)
print('===============Random Forest Performance====================')

print('F1 score: %.3f' % (mean(score)))
print('STD F1 Score: %.3f' % (std(score)))

rc=cross_val_score(classifier, X, y, scoring='recall', cv=cv, n_jobs=-1)
print('Recall: %.3f' % (mean(rc)))

from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score

# Define the specificity scorer
scorer = make_scorer(specificity_score)

# Calculate the cross-validated specificity score
sp = cross_val_score(classifier, X, y, cv=5, scoring=scorer)
print('Specitifity: %.3f' % (mean(sp)))

auc=cross_val_score(classifier, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('AUC ROC: %.3f' % (mean(auc)))

import math

gmean=rc*sp
gmean=mean(gmean)
sqrtg = math.sqrt(gmean)
print('G-Mean: %.3f' % (sqrtg))
print('======================================================')

===============Random Forest Performance====================
F1 score: 0.918
STD F1 Score: 0.001
Recall: 0.905
Specitifity: 0.872
AUC ROC: 0.973
G-Mean: 0.888


In [ ]:
from sklearn.ensemble import AdaBoostClassifier

classifier = AdaBoostClassifier(n_estimators=100,learning_rate=0.1,algorithm='SAMME.R')

score=cross_val_score(classifier, X, y, scoring='f1', cv=cv, n_jobs=-1)
print('===============ADABoost Performance====================')

print('F1 score: %.3f' % (mean(score)))
print('STD F1 Score: %.3f' % (std(score)))

rc=cross_val_score(classifier, X, y, scoring='recall', cv=cv, n_jobs=-1)
print('Recall: %.3f' % (mean(rc)))

from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score

# Define the specificity scorer
scorer = make_scorer(specificity_score)

# Calculate the cross-validated specificity score
sp = cross_val_score(classifier, X, y, cv=5, scoring=scorer)
print('Specitifity: %.3f' % (mean(sp)))

auc=cross_val_score(classifier, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('AUC ROC: %.3f' % (mean(auc)))

import math

gmean=rc*sp
gmean=mean(gmean)
sqrtg = math.sqrt(gmean)
print('G-Mean: %.3f' % (sqrtg))
print('======================================================')